In [1]:
from functions_for_all_nutrients import *
from iron_hemoglobin_functions import *
import matplotlib.backends.backend_pdf
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
username = !whoami
username

['alibow']

In [3]:
index_cols=['location_id','sex_id','age_group_id']

age_group_ids = list(range(7,16))
sex_ids = [2]
coverage_levels = [0.2,0.5,0.8,1]
years = [2021,2022,2023,2024,2025]

In [4]:
coverage_data_dir = f'/ihme/homes/{username[0]}/notebooks/vivarium_research_lsff/data_prep/outputs/waterfall_coverage_all_vehicles.csv'

In [5]:
location_ids = (list(
                pd.read_csv(coverage_data_dir)
                .location_id
                .unique()))
location_ids

[168,
 161,
 201,
 205,
 202,
 6,
 171,
 141,
 179,
 207,
 163,
 11,
 180,
 181,
 184,
 15,
 164,
 213,
 214,
 165,
 196,
 522,
 190,
 189,
 20]

In [6]:
nutrient = 'iron'
#vehicles = ['wheat flour', 'maize flour', 'zero wheat flour', 'zero maize flour', 'industry wheat', 'zero industry wheat']
vehicles = ['industry wheat','zero industry wheat']

effective_fractions = [1] * len(age_group_ids)

In [7]:
baseline_coverage, counterfactual_coverage = get_baseline_and_counterfactual_coverage(coverage_data_dir,
                                             location_ids,
                                             nutrient,
                                             vehicles,
                                             years,
                                             coverage_levels, 'WRA')
counterfactual_coverage.head()

Excluded location IDs [] due to missing data


KeyboardInterrupt: 

# HEMOGLOBIN

In [ ]:
from scipy.stats import norm

# mean and 0.975-quantile of normal distribution for mean difference (MD)
mean = 3
q_975 = 6.1

# 0.975-quantile of standard normal distribution (=1.96, approximately)
q_975_stdnorm = norm().ppf(0.975)

std = (q_975 - mean) / q_975_stdnorm # std dev of normal distribution

# Frozen normal distribution for MD, representing uncertainty in our effect size
hb_md_distribution = norm(mean, std)

In [ ]:
mean_difference_hemoglobin_fort = generate_normal_rr_deficiency_nofort_draws(mean, std, location_ids)
mean_difference_hemoglobin_fort.head()

In [ ]:
# NOTE:
# This approach currently does not consider that new coverage
# applies to those who are slightly more anemic
# than the overall population due to their lack
# of access to fortification

effective_baseline_coverage = get_effective_iron_hemoglobin_coverage(baseline_coverage.set_index(['location_id','vehicle','year']), 
                                                                     sex_ids, 
                                                                     age_group_ids, 
                                                                     effective_fractions, 
                                                                     years)
effective_counterfactual_coverage = get_effective_iron_hemoglobin_coverage(counterfactual_coverage.set_index(['location_id','vehicle','year','coverage_level']), 
                                                                     sex_ids, 
                                                                     age_group_ids, 
                                                                     effective_fractions, 
                                                                     years)

delta_effective_coverage = effective_counterfactual_coverage - effective_baseline_coverage
delta_effective_coverage

In [ ]:
mean_hgb = generate_hemoglobin_values(delta_effective_coverage, 
                                      mean_difference_hemoglobin_fort,
                                      location_ids, age_group_ids, sex_ids)
mean_hgb = mean_hgb.reset_index()
mean_hgb = (mean_hgb
            .loc[mean_hgb.year_id.isin([2025, np.nan])]
            .set_index([c for c in mean_hgb.columns if 'draw' not in c]))

mean_hgb.to_csv('anemia_files_wra/mean_hgb_wra.csv')
mean_hgb = (mean_hgb.reset_index()
            .rename(columns={'year_id':'year'})
            .set_index(['location_id','vehicle','sex_id','age_group_id','coverage_level','year']))

# note, expect to see missing results for locations excluded due to illogical values

mean_hgb

In [ ]:
hgb_sd = get_draws('modelable_entity_id',
                10488,
                source='epi',
                location_id=location_ids,
                age_group_id=age_group_ids,
                sex_id=sex_ids,
                year_id=2019,
                gbd_round_id=6,
                decomp_step='step4',
                status='best')
hgb_sd = (hgb_sd.set_index(['location_id','sex_id','age_group_id'])
 .filter([c for c in hgb_sd if 'draw' in c])
 .reset_index())
hgb_sd.to_csv('anemia_files_wra/sd_hgb_wra.csv')
hgb_sd.head()

In [ ]:
# get age-specific fertility rate
get_covariate_estimates(13, 
                               location_id=location_ids,
                               age_group_id=age_group_ids,
                               sex_id=sex_ids,
                               year_id=2019,
                               gbd_round_id=6,
                               decomp_step='step4').to_csv('anemia_files_wra/asfr.csv')


# get still birth to live birth ratio (not age-specific)
still = get_covariate_estimates(2267, 
                                location_id=location_ids,
                                year_id=2019,
                                gbd_round_id=6,
                                decomp_step='step4').to_csv('anemia_files_wra/still.csv')

In [ ]:
# NOW RUN HEMOGLOBIN_TO_ANEMIA.R FILE BEFORE RUNNING NEXT CELL
assert "Did you run the R file?" == "Double check :)", "Make sure you run the R file!"

In [ ]:
anemia = load_anemia_prev_and_calculate_ylds('anemia_files_wra/anemia_prev_wra.csv')
baseline_anemia = (duplicate_over_simulation_years(anemia.loc[anemia.coverage_level=='baseline'], years)
                   .drop(columns='coverage_level')
                   .set_index(['location_id','age_group_id','sex_id','year','draw','vehicle']))
counterfactual_anemia = (anemia.loc[anemia.coverage_level != 'baseline']
                         .set_index(['location_id','age_group_id','sex_id','year','draw','vehicle','coverage_level']))
averted_anemia = baseline_anemia - counterfactual_anemia
averted_anemia

In [ ]:
anemia_cols = ['mild','moderate','severe','anemic','moderate_ylds','mild_ylds','severe_ylds','anemic_ylds']

anemia_diff = anemia.drop(columns='year').dropna()
pop = get_population(location_id=location_ids,
                    sex_id=sex_ids,
                    age_group_id=age_group_ids,
                    year_id=2019,
                    gbd_round_id=6,
                    decomp_step='step4')
anemia_diff = anemia_diff.merge(pop, on=['location_id','age_group_id','sex_id'])
for col in anemia_cols:
    anemia_diff[f'{col}'] = anemia_diff[f'{col}'] * anemia_diff['population']
anemia_diff = anemia_diff.groupby(['location_id','draw','vehicle','coverage_level']).sum().filter(anemia_cols)#.reset_index()
anemia_diff = anemia_diff.stack().reset_index().rename(columns={'level_4':'measure',0:'value'})
anemia_diff = pd.pivot_table(anemia_diff, index=['location_id','vehicle','coverage_level','measure'],
                            columns='draw',
                            values='value')

anemia_diff.to_pickle('results_raw/iron_anemia_diff_wra.pkl')
anemia_diff.head()

In [ ]:
counts, rates = population_weight_values(averted_anemia.reset_index(), age_group_ids, sex_ids, location_ids)

In [ ]:
counts = summarize_data(counts)
rates = summarize_data(rates)

In [ ]:
anemia_counts_averted = (counts.loc[counts.severity=='anemic']
     .loc[counts.measure=='ylds']
     .set_index(['location_id','vehicle','coverage_level','year'])
     .drop(columns=['severity','measure']))
anemia_counts_averted.head()

In [ ]:
anemia_rates_averted = (rates.loc[rates.severity=='anemic']
     .loc[rates.measure=='ylds']
     .set_index(['location_id','vehicle','coverage_level','year'])
     .drop(columns=['severity','measure']))
anemia_rates_averted.head()

In [ ]:
# total iron responsive anemia burden
iron_responsive_anemia_sequelae = [1004, 1005, 1006, 1008, 1009, 1010, 1012, 1013, 
                                   1014, 1016, 1017, 1018, 1020, 1021, 1022, 1024, 1025, 1026, 
                                   1028, 1029, 1030, 1032, 1033, 1034, 1361, 1364, 1367, 1373, 1376, 
                                   1379, 1385, 1388, 1391, 1397, 1400, 1403, 1409, 1412, 1415, 1421, 
                                   1424, 1427, 1433, 1436, 1439, 1445, 1448, 1451, 5213, 5216, 5219, 
                                   5222, 5225, 5228, 5237, 5240, 5243, 5246, 5249, 5252, 5261, 5264, 
                                   5267, 5270, 5273, 5276, 4985, 4988, 4991, 4994, 4997, 5000, 5009, 
                                   5012, 5015, 5678, 5681, 5684, 7214, 7217, 7220, 4952, 4955, 4958, 
                                   4961, 4964, 4967, 4976, 4979, 4982, 5627, 5630, 5633, 7202, 7205, 
                                   7208, 5393, 5396, 5399, 182, 183, 184, 240, 241, 242, 177, 178, 
                                   179, 144,145,146,172,173,174,525,526,527,1106,1107,1108,537,538,
                                   539,206,207,208, 22989, 22990, 22991, 22992, 22993, 22999, 23000, 
                                   23001, 23002, 23003, 23009, 23010, 23011, 23012, 23013,
                                   5567, 5570, 5573, 5579, 5582, 5585,
                                   23030, 23031, 23032, 23034, 23035, 23036, 23038, 23039, 23040,
                                   23042, 23043, 23044, 23046, 23047, 23048]

ira_ylds = get_draws('sequela_id', iron_responsive_anemia_sequelae, 
                 source='como',
                 location_id=location_ids, 
                 age_group_id=age_group_ids,
                 sex_id=sex_ids,
                 year_id=2019,
                 measure_id=3,
                 decomp_step='step5',
                 gbd_round_id=6)
ira_ylds = ira_ylds.groupby(['location_id','sex_id','age_group_id'], as_index=False).sum()

pop = get_population(location_id=location_ids,
                    age_group_id=age_group_ids,
                    sex_id=sex_ids,
                    gbd_round_id=6,
                    year_id=2019,
                    decomp_step='step4')

ira_ylds = ira_ylds.merge(pop, on=['location_id','sex_id','age_group_id'])
for i in list(range(0,1000)):
    ira_ylds[f'draw_{i}'] = ira_ylds[f'draw_{i}'] * ira_ylds['population']
ira_ylds = ira_ylds.groupby(['location_id']).sum()
#for i in list(range(0,1000)):
#    ira_ylds[f'draw_{i}'] = ira_ylds[f'draw_{i}'] / ira_ylds['population'] * 100_000
ira_ylds = ira_ylds.drop(columns=[c for c in ira_ylds.columns if 'draw' not in c])
ira_ylds.head()

In [ ]:
ira_prev = get_draws('sequela_id', iron_responsive_anemia_sequelae, 
                 source='como',
                 location_id=location_ids, 
                 age_group_id=age_group_ids,
                 sex_id=sex_ids,
                 year_id=2019,
                 measure_id=5,
                 decomp_step='step5',
                 gbd_round_id=6)
ira_prev = ira_prev.groupby(['location_id','sex_id','age_group_id'], as_index=False).sum()

In [ ]:
def calculate_counterfactual_iron_responsive_anemia_prevalence(ira_prevalence_data,
                                                              averted_anemia_data,
                                                              location_ids,
                                                              sex_ids,
                                                              age_group_ids):
    averted_anemia_prepped = averted_anemia_data.filter(['anemic']).reset_index()
    averted_anemia_prepped = averted_anemia_prepped.pivot_table(index=['location_id','age_group_id','sex_id','year','coverage_level','vehicle'],
                                                           columns='draw', values='anemic')
    ira_diff = (ira_prevalence_data
            .set_index(['location_id','age_group_id','sex_id'])
            .drop(columns=['measure_id','sequela_id','year_id','metric_id'])
            - averted_anemia_prepped)
    pop = get_population(location_id=location_ids,
                        age_group_id=age_group_ids,
                        sex_id=sex_ids,
                        year_id=2019,
                        gbd_round_id=6,
                        decomp_step='step4')
    ira_diff = ira_diff.reset_index().merge(pop, on=['location_id','age_group_id','sex_id'])
    for i in list(range(0,1000)):
        ira_diff[f'draw_{i}'] = ira_diff[f'draw_{i}'] * ira_diff['population'] * 100
    ira_diff = ira_diff.groupby(['location_id','year','vehicle','coverage_level']).sum()
    for i in list(range(0,1000)):
        ira_diff[f'draw_{i}'] = ira_diff[f'draw_{i}'] / ira_diff['population']
    ira_diff = ira_diff.filter([c for c in ira_diff if 'draw' in c])
    return ira_diff


In [ ]:
anemia_diff = calculate_counterfactual_anemia_prevalence(location_ids, sex_ids, age_group_ids,
                                              anemia)
ira_diff = calculate_counterfactual_iron_responsive_anemia_prevalence(ira_prev,
                                                              averted_anemia,
                                                              location_ids,
                                                              sex_ids,
                                                              age_group_ids)
anemia_diff['measure'] = 'anemia'
ira_diff['measure'] = 'iron_responsive_anemia'
deficiency_difference = pd.concat([anemia_diff.reset_index(), ira_diff.reset_index()], ignore_index=True, sort=True)
deficiency_difference.to_pickle('results_raw/iron_deficiency_difference_wra.pkl')
deficiency_difference.head()

In [ ]:
anemia_pifs = anemia_counts_averted / (ira_ylds) * 100
anemia_pifs.head()

In [ ]:
anemia_pifs['measure'] = 'pif'
anemia_rates_averted['measure'] = 'rates_averted'
anemia_counts_averted['measure'] = 'counts_averted'

anemia_wra_results = pd.concat([anemia_pifs, anemia_rates_averted, anemia_counts_averted], sort=True).reset_index()
anemia_wra_results = anemia_wra_results.set_index([c for c in anemia_wra_results if 'draw' not in c])

anemia_wra_results.to_pickle('results_raw/iron_anemia_wra.pkl')

anemia_wra_results.head()

# MATERNAL DISORDERS

In [ ]:
rr = get_draws('rei_id',
                 95,
                 source='rr',
                 location_id=location_ids,
                 age_group_id=age_group_ids,
                 sex_id=sex_ids,
                 year_id=2019,
                 gbd_round_id=6,
                 decomp_step='step4'
                 )
rr = rr.loc[rr.cause_id==367].set_index(['sex_id','age_group_id'])
rr = rr.drop(columns=[c for c in rr.columns if 'draw' not in c])
rr

In [ ]:
baseline_hgb = get_draws('modelable_entity_id',
                    10487,
                    source='epi',
                    location_id=location_ids,
                    age_group_id=age_group_ids,
                    sex_id=sex_ids,
                    year_id=2019,
                    gbd_round_id=6,
                    decomp_step='step4',
                    status='best')
baseline_hgb = baseline_hgb.set_index(['location_id','sex_id','age_group_id'])
baseline_hgb = baseline_hgb.drop(columns=[c for c in baseline_hgb.columns if 'draw' not in c])
baseline_hgb.head()

In [ ]:
hgb_diff = (((mean_hgb - baseline_hgb) / 10)
            .reset_index()
            .set_index(['sex_id','age_group_id','location_id','coverage_level','year','vehicle']))
hgb_diff.head()

In [ ]:
import math
frac_reduction = ((1 / math.e ** (np.log(rr) * hgb_diff))
                  .reset_index()
                  .set_index(['location_id','sex_id','age_group_id','coverage_level','year','vehicle']))
frac_reduction.head()

In [ ]:
maternal_ylds = get_draws('cause_id',
                367,
                source='como',
                location_id=location_ids,
                age_group_id=age_group_ids,
                measure_id=3, #ylds
                metric_id=3, #rate
                sex_id=sex_ids,
                year_id=2019,
                gbd_round_id=6,
                decomp_step='step5')
pop = get_population(location_id=location_ids,
                    sex_id=sex_ids,
                    age_group_id=age_group_ids,
                    gbd_round_id=6,
                    decomp_step='step4')
maternal_ylds = maternal_ylds.merge(pop, on=['location_id','sex_id','age_group_id'])
for i in list(range(0,1000)):
    maternal_ylds[f'draw_{i}'] = maternal_ylds[f'draw_{i}'] * maternal_ylds['population']

maternal_ylds = maternal_ylds.set_index(['location_id','sex_id','age_group_id'])
maternal_ylds = maternal_ylds.drop(columns=[c for c in maternal_ylds.columns if 'draw' not in c])
maternal_ylds.head()

In [ ]:
maternal_ylls = get_draws('cause_id',
                367,
                source='codcorrect',
                location_id=location_ids,
                age_group_id=age_group_ids,
                measure_id=4,
                metric_id=1,
                sex_id=sex_ids,
                year_id=2019,
                gbd_round_id=6,
                decomp_step='step5',
                status='latest')

maternal_ylls = maternal_ylls.set_index(['location_id','sex_id','age_group_id'])
maternal_ylls = maternal_ylls.drop(columns=[c for c in maternal_ylls.columns if 'draw' not in c])
maternal_ylls.head()

In [ ]:
maternal_dalys = maternal_ylds + maternal_ylls
maternal_dalys.head()

In [ ]:
counterfactual_maternal_dalys = maternal_dalys * frac_reduction
counterfactual_maternal_dalys = (counterfactual_maternal_dalys.groupby(['location_id','vehicle','coverage_level','year']).sum())
maternal_counts_averted = maternal_dalys.groupby('location_id').sum() - counterfactual_maternal_dalys
maternal_counts_averted.head()

In [ ]:
maternal_rates_averted = (maternal_counts_averted.reset_index()
                 .merge(pop.groupby('location_id', as_index=False).sum().drop(columns='year_id'), 
                        on='location_id'))
for i in list(range(0,1000)):
    maternal_rates_averted[f'draw_{i}'] = (maternal_rates_averted[f'draw_{i}'] / maternal_rates_averted['population']) * 100_000
maternal_rates_averted = maternal_rates_averted.set_index(['location_id','vehicle','coverage_level','year'])
maternal_rates_averted = maternal_rates_averted.drop(columns=[c for c in maternal_rates_averted.columns if 'draw' not in c])
maternal_rates_averted.head()

In [ ]:
iron_attributable_maternal_disorder_burden = get_draws(gbd_id_type=['rei_id','cause_id'],
                                                      gbd_id=[95,367],
                                                      location_id=location_ids,
                                                      sex_id=sex_ids,
                                                      age_group_id=age_group_ids,
                                                      measure_id=2, # dalys
                                                      metric_id=1, # number
                                                      source='burdenator',
                                                      decomp_step='step5',
                                                      gbd_round_id=6,
                                                      year_id=2019)
iron_attributable_maternal_disorder_burden = iron_attributable_maternal_disorder_burden.groupby('location_id').sum()
iron_attributable_maternal_disorder_burden = iron_attributable_maternal_disorder_burden.drop(columns=[c for c in iron_attributable_maternal_disorder_burden.columns if 'draw' not in c])
#iron_attributable_maternal_disorder_burden = (iron_attributable_maternal_disorder_burden.reset_index()
#                                              .merge(pop.groupby('location_id').sum().reset_index(),
#                                                     on='location_id'))
#for i in list(range(0,1000)):
#    iron_attributable_maternal_disorder_burden[f'draw_{i}'] = iron_attributable_maternal_disorder_burden[f'draw_{i}'] / iron_attributable_maternal_disorder_burden['population']
#iron_attributable_maternal_disorder_burden = iron_attributable_maternal_disorder_burden.set_index(['location_id'])
iron_attributable_maternal_disorder_burden = iron_attributable_maternal_disorder_burden.drop(columns=[c for c in iron_attributable_maternal_disorder_burden.columns if 'draw' not in c])

iron_attributable_maternal_disorder_burden

In [ ]:
iron_attributable_maternal_disorder_burden.to_pickle('results_raw/iron_maternal_dalys.pkl')

In [ ]:
ira_ylds.to_pickle('results_raw/wra_ira_dalys.pkl')

In [ ]:
maternal_disorders_pif = maternal_counts_averted / (iron_attributable_maternal_disorder_burden) * 100
maternal_disorders_pif.head()

In [ ]:
maternal_disorders_pif['measure'] = 'pif'
maternal_rates_averted['measure'] = 'rates_averted'
maternal_counts_averted['measure'] = 'counts_averted'

maternal_disorders_wra_results = pd.concat([maternal_disorders_pif, maternal_rates_averted, maternal_counts_averted], sort=True).reset_index()
maternal_disorders_wra_results = maternal_disorders_wra_results.set_index([c for c in maternal_disorders_wra_results if 'draw' not in c])

maternal_disorders_wra_results.to_pickle('results_raw/iron_maternal_disorders_wra.pkl')

maternal_disorders_wra_results.head()

# OVERALL

In [ ]:
overall_counts_averted = anemia_counts_averted + maternal_counts_averted
overall_rates_averted = anemia_rates_averted + maternal_rates_averted
overall_counts_averted.head()

In [ ]:
overall_pif = overall_counts_averted / ((ira_ylds + iron_attributable_maternal_disorder_burden)) * 100
overall_pif.head()

In [ ]:
overall_pif['measure'] = 'pif'
overall_counts_averted['measure'] = 'counts_averted'
overall_rates_averted['measure'] = 'rates_averted'
iron_wra_results = pd.concat([overall_pif.reset_index(),
                         overall_counts_averted.reset_index(),
                         overall_rates_averted.reset_index()],
                        ignore_index=True)
iron_wra_results.to_pickle('results_raw/iron_wra_industry_wheat_update.pkl')

In [ ]:
test = overall_pif.mean(axis=1).reset_index()
test.loc[test.location_id==214]